<a href="https://www.kaggle.com/code/shivamsourav2002/bert-nlp-12?scriptVersionId=172662896" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# EXPERIMENT NO 11
EXTEND THE NER EXPERIMENT TO INCLUDE NAMED ENTITY LINKING ASSOCIATED RECOGNIZED ENTITIES WITH THEIR CORRESPONDING DATABASE 

# EXPERIMENT NO 12
Use pre-trained BERT models for various NLP tasks like question answering , sentiment analysis and text classification. Fine tune BERT for a specific task and compare its performance with traditional models.

In [5]:
import torch 
from transformers import BertTokenizer,BertForSequenceClassification
from torch.nn import functional as F

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)

sentiment_labels = {0:'Negative',1:'Neutral',2:'Positive'}
def predict_sentiment(text):
    inputs = tokenizer(text,return_tensors='pt',padding=True,truncation=True)
    outputs = model(**inputs)
    logits = outputs.logits
    probabilities = F.softmax(logits,dim=1)
    
    predicted_label = torch.argmax(probabilities,dim=1).item()
    predicted_sentiment = sentiment_labels[predicted_label]
    return predicted_sentiment,probabilities

text =  """I didn't enjoyed the movie, it was very horrible"""
predicted_sentiment,probabilities = predict_sentiment(text)
print(f"Predicted sentiment : {predicted_sentiment} with confidence of {probabilities}%")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Predicted sentiment : Neutral with confidence of tensor([[0.3082, 0.6918]], grad_fn=<SoftmaxBackward0>)%


In [6]:
from transformers import pipeline
sa = pipeline('text-classification', model=model,tokenizer=tokenizer)
sa("""The movie was very Good , I liked it too much.""")

2024-04-18 10:32:12.418024: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 10:32:12.418129: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 10:32:12.696894: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[{'label': 'LABEL_1', 'score': 0.6913214325904846}]

In [7]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Load the dataset
data = pd.read_csv('/kaggle/input/spam-mails-dataset/spam_ham_dataset.csv')
data = data[:int(data.shape[0]*0.2)]
test = data[int(data.shape[0]*0.2):]


# Encode the text data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
encoded_data = tokenizer.batch_encode_plus(
    data['text'].values,
    add_special_tokens=True,
    return_attention_mask=True,
    padding='max_length',
    max_length=512,
    return_tensors='pt',
    truncation=True
)

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['label'].values)
labels = torch.tensor(labels)

# Create input tensors
input_ids = encoded_data['input_ids']
attention_masks = encoded_data['attention_mask']

# Create the dataset and dataloader
dataset = TensorDataset(input_ids, attention_masks, labels)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

# Load the pre-trained BERT model
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(set(data['label'])),  # Number of unique labels
    output_attentions=False,
    output_hidden_states=False
)

# Set the device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Define the optimizer and loss function
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in dataloader:
        # Unpack the batch
        b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

        # Forward pass
        outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}/{epochs}, Loss: {loss.item()}')

# Evaluation
model.eval()
eval_accuracy = 0
eval_data = TensorDataset(input_ids, attention_masks, labels)
eval_dataloader = DataLoader(eval_data, batch_size=16)

for batch in eval_dataloader:
    b_input_ids, b_attention_mask, b_labels = tuple(t.to(device) for t in batch)

    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_attention_mask)[0]

    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    eval_accuracy += (logits.argmax(axis=-1) == label_ids).mean()

eval_accuracy /= len(eval_dataloader)
print(f'Evaluation Accuracy: {eval_accuracy*100:.2f}%')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3, Loss: 0.08677767217159271
Epoch 2/3, Loss: 0.023762719705700874
Epoch 3/3, Loss: 0.00340729346498847
Evaluation Accuracy: 100.00%


In [11]:
text = test.iloc[0]['text']

In [12]:
text

'Subject: calpine daily gas nomination\r\n>\r\nricky a . archer\r\nfuel supply\r\n700 louisiana , suite 2700\r\nhouston , texas 77002\r\n713 - 830 - 8659 direct\r\n713 - 830 - 8722 fax\r\n- calpine daily gas nomination 1 . doc'

In [13]:
# Function to perform inference on a single input string
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
def predict(input_text):
    # Encode the input text
    encoded_input = tokenizer.encode_plus(
        input_text,
        add_special_tokens=True,
        return_attention_mask=True,
        padding='max_length',
        max_length=512,
        return_tensors='pt'
    )

    input_ids = encoded_input['input_ids'].to(device)
    attention_mask = encoded_input['attention_mask'].to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Get the predicted label
    predicted_label_idx = logits.argmax().item()
    predicted_label = label_encoder.inverse_transform([predicted_label_idx])

    return predicted_label[0]
# Get input text from the user
input_text = input("Enter your text: ")

# Perform inference and print the predicted label
predicted_label = predict(input_text)
print(f"Predicted label: {predicted_label}")

Enter your text:  Subject: calpine daily gas nomination\r\n>\r\nricky a . archer\r\nfuel supply\r\n700 louisiana , suite 2700\r\nhouston , texas 77002\r\n713 - 830 - 8659 direct\r\n713 - 830 - 8722 fax\r\n- calpine daily gas nomination 1 . doc


Predicted label: ham
